# Data Scraping - Mining for GOLD

Scraping websites to gather data that may be useful for the model will prove more fruitful than relying on the Kaggle files which we know have incomplete data.

Sports-Reference.com is a treasure trove.

Scraping also gives us the opportunity to avail of data that is not in the Kaggle dataset, e.g. the Strength of Schedule rating for each team.

> **Strength of Schedule:** A rating of strength of schedule. 
> The rating is denominated in points above/below average, where zero is average. Non-Division I games are excluded from the ratings.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import time
import pickle
import os
import requests
from bs4 import BeautifulSoup

## Season Stats for Team

There is a summary page containing the season stats for each school team. The URL for the page has the following format:

    https://www.sports-reference.com/cbb/seasons/[SEASON]-school-stats.html

where [SEASON] is the year the season ends. For example the stats for the 2018/2019 season are found on the following URL:

    https://www.sports-reference.com/cbb/seasons/2019-school-stats.html

This will get us to the correct page and then it is a matter of identifying the location of the data we're interested in scraping on the page.

In [2]:
season = 2019

In [3]:
url = 'https://www.sports-reference.com/cbb/seasons/{}-school-stats.html#basic_school_stats::none'.format(season)

df = pd.read_html(url)[0]

df.head()

Unnamed: 0_level_0      Unnamed: 1_level_0 Overall                        \
                  Rk                  School       G   W   L  W-L%     SRS   
0                  1  Abilene Christian NCAA      34  27   7  .794   -1.91   
1                  2               Air Force      32  14  18  .438   -4.28   
2                  3                   Akron      33  17  16  .515    4.86   
3                  4             Alabama A&M      32   5  27  .156  -19.23   
4                  5      Alabama-Birmingham      35  20  15  .571    0.36   

         Conf.      ... School Totals                                       \
     SOS     W   L  ...            FT  FTA   FT%  ORB   TRB  AST  STL  BLK   
0  -7.34    14   4  ...           457  642  .712  325  1110  525  297   93   
1   0.24     8  10  ...           341  503  .678  253  1077  434  154   57   
2   1.09     8  10  ...           380  539  .705  312  1204  399  185  106   
3  -8.38     4  14  ...           284  453  .627  314  1032  385  234   50   
4  -1.52    10   8  ...           424  630  .673  367  1279  401  218   82   

             
   TOV   PF  
0  407  635  
1  423  543  
2  388  569  
3  487  587  
4  399  578  

[5 rows x 34 columns]

## Cleaning the data

The first thing we will need to do is fix the scraped header of the table so it's a single tier.

In [4]:
# Fix the header
header = df.columns.tolist()
cols = [col[1].lower() for col in header]
df.columns = cols

# Look at more of the table
df.head(30)

,rk,school,g,w,l,w-l%,srs,sos,w,l,...,ft,fta,ft%,orb,trb,ast,stl,blk,tov,pf
0,1,Abilene Christian NCAA,34,27,7,.794,-1.91,-7.34,14,4,...,457,642,.712,325,1110,525,297,93,407,635
1,2,Air Force,32,14,18,.438,-4.28,0.24,8,10,...,341,503,.678,253,1077,434,154,57,423,543
2,3,Akron,33,17,16,.515,4.86,1.09,8,10,...,380,539,.705,312,1204,399,185,106,388,569
3,4,Alabama A&M,32,5,27,.156,-19.23,-8.38,4,14,...,284,453,.627,314,1032,385,234,50,487,587
4,5,Alabama-Birmingham,35,20,15,.571,0.36,-1.52,10,8,...,424,630,.673,367,1279,401,218,82,399,578
5,6,Alabama State,31,12,19,.387,-15.60,-7.84,9,9,...,446,684,.652,365,1094,313,203,102,451,565
6,7,Alabama,34,18,16,.529,9.45,9.01,8,10,...,492,739,.666,384,1285,418,157,160,465,572
7,8,Albany (NY),32,12,20,.375,-9.38,-6.70,7,9,...,420,564,.745,294,1081,402,195,78,454,566
8,9,Alcorn State,31,10,21,.323,-22.08,-8.97,6,12,...,366,543,.674,334,1079,391,229,107,492,548
9,10,American,30,15,15,.500,-4.19,-7.23,9,9,...,411,589,.698,251,1014,402,221,131,386,520


When we look at more of the data we see that the header is repeated across two rows every so often. We can identify these rows by the 'rk' column and then remove them from the data.

In [5]:
df[(df['rk'] == 'Rk') | (df['rk'] != df['rk'])]
# These rows need to be removed

,rk,school,g,w,l,w-l%,srs,sos,w,l,...,ft,fta,ft%,orb,trb,ast,stl,blk,tov,pf
20,NaN,NaN,Overall,Overall,Overall,Overall,Overall,Overall,Conf.,Conf.,...,School Totals,School Totals,School Totals,School Totals,School Totals,School Totals,School Totals,School Totals,School Totals,School Totals
21,Rk,School,G,W,L,W-L%,SRS,SOS,W,L,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
42,NaN,NaN,Overall,Overall,Overall,Overall,Overall,Overall,Conf.,Conf.,...,School Totals,School Totals,School Totals,School Totals,School Totals,School Totals,School Totals,School Totals,School Totals,School Totals
43,Rk,School,G,W,L,W-L%,SRS,SOS,W,L,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
64,NaN,NaN,Overall,Overall,Overall,Overall,Overall,Overall,Conf.,Conf.,...,School Totals,School Totals,School Totals,School Totals,School Totals,School Totals,School Totals,School Totals,School Totals,School Totals
65,Rk,School,G,W,L,W-L%,SRS,SOS,W,L,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
86,NaN,NaN,Overall,Overall,Overall,Overall,Overall,Overall,Conf.,Conf.,...,School Totals,School Totals,School Totals,School Totals,School Totals,School Totals,School Totals,School Totals,School Totals,School Totals
87,Rk,School,G,W,L,W-L%,SRS,SOS,W,L,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
108,NaN,NaN,Overall,Overall,Overall,Overall,Overall,Overall,Conf.,Conf.,...,School Totals,School Totals,School Totals,School Totals,School Totals,School Totals,School Totals,School Totals,School Totals,School Totals
109,Rk,School,G,W,L,W-L%,SRS,SOS,W,L,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF


In [6]:
# test run
df[(df['rk'] != 'Rk') & (df['rk'] == df['rk'])]


,rk,school,g,w,l,w-l%,srs,sos,w,l,...,ft,fta,ft%,orb,trb,ast,stl,blk,tov,pf
0,1,Abilene Christian NCAA,34,27,7,.794,-1.91,-7.34,14,4,...,457,642,.712,325,1110,525,297,93,407,635
1,2,Air Force,32,14,18,.438,-4.28,0.24,8,10,...,341,503,.678,253,1077,434,154,57,423,543
2,3,Akron,33,17,16,.515,4.86,1.09,8,10,...,380,539,.705,312,1204,399,185,106,388,569
3,4,Alabama A&M,32,5,27,.156,-19.23,-8.38,4,14,...,284,453,.627,314,1032,385,234,50,487,587
4,5,Alabama-Birmingham,35,20,15,.571,0.36,-1.52,10,8,...,424,630,.673,367,1279,401,218,82,399,578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,349,Wright State,35,21,14,.600,3.29,-0.89,13,5,...,510,692,.737,382,1229,484,214,72,402,545
383,350,Wyoming,32,8,24,.250,-9.75,0.19,4,14,...,477,660,.723,167,983,331,176,88,450,588
384,351,Xavier,35,19,16,.543,9.61,8.06,9,9,...,437,644,.679,371,1281,519,190,128,450,550
385,352,Yale NCAA,30,22,8,.733,5.52,-1.24,10,4,...,411,557,.738,259,1157,503,177,131,392,510


This looks good so we can apply the change to the dataframe.

In [9]:
df = df[(df['rk'] != 'Rk') & (df['rk'] == df['rk'])]
df.head()

,rk,school,g,w,l,w-l%,srs,sos,w,l,...,ft,fta,ft%,orb,trb,ast,stl,blk,tov,pf
0,1,Abilene Christian NCAA,34,27,7,.794,-1.91,-7.34,14,4,...,457,642,.712,325,1110,525,297,93,407,635
1,2,Air Force,32,14,18,.438,-4.28,0.24,8,10,...,341,503,.678,253,1077,434,154,57,423,543
2,3,Akron,33,17,16,.515,4.86,1.09,8,10,...,380,539,.705,312,1204,399,185,106,388,569
3,4,Alabama A&M,32,5,27,.156,-19.23,-8.38,4,14,...,284,453,.627,314,1032,385,234,50,487,587
4,5,Alabama-Birmingham,35,20,15,.571,0.36,-1.52,10,8,...,424,630,.673,367,1279,401,218,82,399,578


## Repurpose the school name

Transform the school column to something usable for scraping the website

In [10]:
df.school.unique()

array(['Abilene Christian\xa0NCAA', 'Air Force', 'Akron', 'Alabama A&M',
       'Alabama-Birmingham', 'Alabama State', 'Alabama', 'Albany (NY)',
       'Alcorn State', 'American', 'Appalachian State',
       'Arizona State\xa0NCAA', 'Arizona', 'Little Rock',
       'Arkansas-Pine Bluff', 'Arkansas State', 'Arkansas', 'Army',
       'Auburn\xa0NCAA', 'Austin Peay', 'Ball State', 'Baylor\xa0NCAA',
       'Belmont\xa0NCAA', 'Bethune-Cookman', 'Binghamton', 'Boise State',
       'Boston College', 'Boston University', 'Bowling Green State',
       'Bradley\xa0NCAA', 'Brigham Young', 'Brown', 'Bryant', 'Bucknell',
       'Buffalo\xa0NCAA', 'Butler', 'Cal Poly', 'Cal State Bakersfield',
       'Cal State Fullerton', 'Cal State Northridge',
       'California Baptist', 'UC-Davis', 'UC-Irvine\xa0NCAA',
       'UC-Riverside', 'UC-Santa Barbara', 'University of California',
       'Campbell', 'Canisius', 'Central Arkansas',
       'Central Connecticut State', 'Central Florida\xa0NCAA',
       'Ce

In [11]:
# Step 1 : lower case
school_1 = [s.lower() for s in df.school.unique()]
school_1[1]

'air force'

In [12]:
# Step 2 : Strip punctuation
# https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string

unwanted = "!#$%&'()*+,./:;<=>?@[\]^_`{|}~"

school_2 = [s.translate(str.maketrans('', '', unwanted)) for s in school_1]

In [13]:
# Step 3 : Get rid of the '\xa0ncaa' nonsense
school_3 = [s.replace('\xa0ncaa', '') for s in school_2]

In [14]:
school_4 = [s.replace(' ', '-') for s in school_3]
school_4

['abilene-christian',
 'air-force',
 'akron',
 'alabama-am',
 'alabama-birmingham',
 'alabama-state',
 'alabama',
 'albany-ny',
 'alcorn-state',
 'american',
 'appalachian-state',
 'arizona-state',
 'arizona',
 'little-rock',
 'arkansas-pine-bluff',
 'arkansas-state',
 'arkansas',
 'army',
 'auburn',
 'austin-peay',
 'ball-state',
 'baylor',
 'belmont',
 'bethune-cookman',
 'binghamton',
 'boise-state',
 'boston-college',
 'boston-university',
 'bowling-green-state',
 'bradley',
 'brigham-young',
 'brown',
 'bryant',
 'bucknell',
 'buffalo',
 'butler',
 'cal-poly',
 'cal-state-bakersfield',
 'cal-state-fullerton',
 'cal-state-northridge',
 'california-baptist',
 'uc-davis',
 'uc-irvine',
 'uc-riverside',
 'uc-santa-barbara',
 'university-of-california',
 'campbell',
 'canisius',
 'central-arkansas',
 'central-connecticut-state',
 'central-florida',
 'central-michigan',
 'charleston-southern',
 'charlotte',
 'chattanooga',
 'chicago-state',
 'cincinnati',
 'citadel',
 'clemson',
 'cleve

Now we have a pretty nice list of URL friendly strings and we can turn the steps to compose the list into a function for later re-use.

In [15]:
def transform_school_name(s):
    unwanted = "!#$%&'()*+,./:;<=>?@[\]^_`{|}~"
    return (s.translate(str.maketrans('', '', unwanted))
             .lower()
             .replace('ncaa', '')
             .replace(' ', '-')
             .strip())


The function will more than likely need to be tweaked later because there are bound to be exceptions. In the meantime we can apply it to the 'school' column to create a new column 'school_formatted'. We'll be able to use this column for scraping the school pages of the website.

In [16]:
df["school_formatted"] = df.school.apply(transform_school_name)
df.head()

,rk,school,g,w,l,w-l%,srs,sos,w,l,...,fta,ft%,orb,trb,ast,stl,blk,tov,pf,school_formatted
0,1,Abilene Christian NCAA,34,27,7,.794,-1.91,-7.34,14,4,...,642,.712,325,1110,525,297,93,407,635,abilene-christian
1,2,Air Force,32,14,18,.438,-4.28,0.24,8,10,...,503,.678,253,1077,434,154,57,423,543,air-force
2,3,Akron,33,17,16,.515,4.86,1.09,8,10,...,539,.705,312,1204,399,185,106,388,569,akron
3,4,Alabama A&M,32,5,27,.156,-19.23,-8.38,4,14,...,453,.627,314,1032,385,234,50,487,587,alabama-am
4,5,Alabama-Birmingham,35,20,15,.571,0.36,-1.52,10,8,...,630,.673,367,1279,401,218,82,399,578,alabama-birmingham


### Testing the function

Let's lift the URLs for the college specific pages from this page:

    https://www.sports-reference.com/cbb/seasons/2019-school-stats.html

Open the link in the brower and run this code in the JavaScript console:

    l=[]; document.querySelector('#div_basic_school_stats').querySelectorAll('a').forEach((a)=>l.push(a.href)); l.join('\n');

This task could have been completed with Python but would have been more cumbersome.

Now we have a list of URLs to test the function against.

In [17]:
browser_links = """https://www.sports-reference.com/cbb/schools/abilene-christian/2019.html
https://www.sports-reference.com/cbb/schools/air-force/2019.html
https://www.sports-reference.com/cbb/schools/akron/2019.html
https://www.sports-reference.com/cbb/schools/alabama-am/2019.html
https://www.sports-reference.com/cbb/schools/alabama-birmingham/2019.html
https://www.sports-reference.com/cbb/schools/alabama-state/2019.html
https://www.sports-reference.com/cbb/schools/alabama/2019.html
https://www.sports-reference.com/cbb/schools/albany-ny/2019.html
https://www.sports-reference.com/cbb/schools/alcorn-state/2019.html
https://www.sports-reference.com/cbb/schools/american/2019.html
https://www.sports-reference.com/cbb/schools/appalachian-state/2019.html
https://www.sports-reference.com/cbb/schools/arizona-state/2019.html
https://www.sports-reference.com/cbb/schools/arizona/2019.html
https://www.sports-reference.com/cbb/schools/arkansas-little-rock/2019.html
https://www.sports-reference.com/cbb/schools/arkansas-pine-bluff/2019.html
https://www.sports-reference.com/cbb/schools/arkansas-state/2019.html
https://www.sports-reference.com/cbb/schools/arkansas/2019.html
https://www.sports-reference.com/cbb/schools/army/2019.html
https://www.sports-reference.com/cbb/schools/auburn/2019.html
https://www.sports-reference.com/cbb/schools/austin-peay/2019.html
https://www.sports-reference.com/cbb/schools/ball-state/2019.html
https://www.sports-reference.com/cbb/schools/baylor/2019.html
https://www.sports-reference.com/cbb/schools/belmont/2019.html
https://www.sports-reference.com/cbb/schools/bethune-cookman/2019.html
https://www.sports-reference.com/cbb/schools/binghamton/2019.html
https://www.sports-reference.com/cbb/schools/boise-state/2019.html
https://www.sports-reference.com/cbb/schools/boston-college/2019.html
https://www.sports-reference.com/cbb/schools/boston-university/2019.html
https://www.sports-reference.com/cbb/schools/bowling-green-state/2019.html
https://www.sports-reference.com/cbb/schools/bradley/2019.html
https://www.sports-reference.com/cbb/schools/brigham-young/2019.html
https://www.sports-reference.com/cbb/schools/brown/2019.html
https://www.sports-reference.com/cbb/schools/bryant/2019.html
https://www.sports-reference.com/cbb/schools/bucknell/2019.html
https://www.sports-reference.com/cbb/schools/buffalo/2019.html
https://www.sports-reference.com/cbb/schools/butler/2019.html
https://www.sports-reference.com/cbb/schools/cal-poly/2019.html
https://www.sports-reference.com/cbb/schools/cal-state-bakersfield/2019.html
https://www.sports-reference.com/cbb/schools/cal-state-fullerton/2019.html
https://www.sports-reference.com/cbb/schools/cal-state-northridge/2019.html
https://www.sports-reference.com/cbb/schools/california-baptist/2019.html
https://www.sports-reference.com/cbb/schools/california-davis/2019.html
https://www.sports-reference.com/cbb/schools/california-irvine/2019.html
https://www.sports-reference.com/cbb/schools/california-riverside/2019.html
https://www.sports-reference.com/cbb/schools/california-santa-barbara/2019.html
https://www.sports-reference.com/cbb/schools/california/2019.html
https://www.sports-reference.com/cbb/schools/campbell/2019.html
https://www.sports-reference.com/cbb/schools/canisius/2019.html
https://www.sports-reference.com/cbb/schools/central-arkansas/2019.html
https://www.sports-reference.com/cbb/schools/central-connecticut-state/2019.html
https://www.sports-reference.com/cbb/schools/central-florida/2019.html
https://www.sports-reference.com/cbb/schools/central-michigan/2019.html
https://www.sports-reference.com/cbb/schools/charleston-southern/2019.html
https://www.sports-reference.com/cbb/schools/charlotte/2019.html
https://www.sports-reference.com/cbb/schools/chattanooga/2019.html
https://www.sports-reference.com/cbb/schools/chicago-state/2019.html
https://www.sports-reference.com/cbb/schools/cincinnati/2019.html
https://www.sports-reference.com/cbb/schools/citadel/2019.html
https://www.sports-reference.com/cbb/schools/clemson/2019.html
https://www.sports-reference.com/cbb/schools/cleveland-state/2019.html
https://www.sports-reference.com/cbb/schools/coastal-carolina/2019.html
https://www.sports-reference.com/cbb/schools/colgate/2019.html
https://www.sports-reference.com/cbb/schools/college-of-charleston/2019.html
https://www.sports-reference.com/cbb/schools/colorado-state/2019.html
https://www.sports-reference.com/cbb/schools/colorado/2019.html
https://www.sports-reference.com/cbb/schools/columbia/2019.html
https://www.sports-reference.com/cbb/schools/connecticut/2019.html
https://www.sports-reference.com/cbb/schools/coppin-state/2019.html
https://www.sports-reference.com/cbb/schools/cornell/2019.html
https://www.sports-reference.com/cbb/schools/creighton/2019.html
https://www.sports-reference.com/cbb/schools/dartmouth/2019.html
https://www.sports-reference.com/cbb/schools/davidson/2019.html
https://www.sports-reference.com/cbb/schools/dayton/2019.html
https://www.sports-reference.com/cbb/schools/delaware-state/2019.html
https://www.sports-reference.com/cbb/schools/delaware/2019.html
https://www.sports-reference.com/cbb/schools/denver/2019.html
https://www.sports-reference.com/cbb/schools/depaul/2019.html
https://www.sports-reference.com/cbb/schools/detroit-mercy/2019.html
https://www.sports-reference.com/cbb/schools/drake/2019.html
https://www.sports-reference.com/cbb/schools/drexel/2019.html
https://www.sports-reference.com/cbb/schools/duke/2019.html
https://www.sports-reference.com/cbb/schools/duquesne/2019.html
https://www.sports-reference.com/cbb/schools/east-carolina/2019.html
https://www.sports-reference.com/cbb/schools/east-tennessee-state/2019.html
https://www.sports-reference.com/cbb/schools/eastern-illinois/2019.html
https://www.sports-reference.com/cbb/schools/eastern-kentucky/2019.html
https://www.sports-reference.com/cbb/schools/eastern-michigan/2019.html
https://www.sports-reference.com/cbb/schools/eastern-washington/2019.html
https://www.sports-reference.com/cbb/schools/elon/2019.html
https://www.sports-reference.com/cbb/schools/evansville/2019.html
https://www.sports-reference.com/cbb/schools/fairfield/2019.html
https://www.sports-reference.com/cbb/schools/fairleigh-dickinson/2019.html
https://www.sports-reference.com/cbb/schools/florida-am/2019.html
https://www.sports-reference.com/cbb/schools/florida-atlantic/2019.html
https://www.sports-reference.com/cbb/schools/florida-gulf-coast/2019.html
https://www.sports-reference.com/cbb/schools/florida-international/2019.html
https://www.sports-reference.com/cbb/schools/florida-state/2019.html
https://www.sports-reference.com/cbb/schools/florida/2019.html
https://www.sports-reference.com/cbb/schools/fordham/2019.html
https://www.sports-reference.com/cbb/schools/fresno-state/2019.html
https://www.sports-reference.com/cbb/schools/furman/2019.html
https://www.sports-reference.com/cbb/schools/gardner-webb/2019.html
https://www.sports-reference.com/cbb/schools/george-mason/2019.html
https://www.sports-reference.com/cbb/schools/george-washington/2019.html
https://www.sports-reference.com/cbb/schools/georgetown/2019.html
https://www.sports-reference.com/cbb/schools/georgia-southern/2019.html
https://www.sports-reference.com/cbb/schools/georgia-state/2019.html
https://www.sports-reference.com/cbb/schools/georgia-tech/2019.html
https://www.sports-reference.com/cbb/schools/georgia/2019.html
https://www.sports-reference.com/cbb/schools/gonzaga/2019.html
https://www.sports-reference.com/cbb/schools/grambling/2019.html
https://www.sports-reference.com/cbb/schools/grand-canyon/2019.html
https://www.sports-reference.com/cbb/schools/green-bay/2019.html
https://www.sports-reference.com/cbb/schools/hampton/2019.html
https://www.sports-reference.com/cbb/schools/hartford/2019.html
https://www.sports-reference.com/cbb/schools/harvard/2019.html
https://www.sports-reference.com/cbb/schools/hawaii/2019.html
https://www.sports-reference.com/cbb/schools/high-point/2019.html
https://www.sports-reference.com/cbb/schools/hofstra/2019.html
https://www.sports-reference.com/cbb/schools/holy-cross/2019.html
https://www.sports-reference.com/cbb/schools/houston-baptist/2019.html
https://www.sports-reference.com/cbb/schools/houston/2019.html
https://www.sports-reference.com/cbb/schools/howard/2019.html
https://www.sports-reference.com/cbb/schools/idaho-state/2019.html
https://www.sports-reference.com/cbb/schools/idaho/2019.html
https://www.sports-reference.com/cbb/schools/illinois-chicago/2019.html
https://www.sports-reference.com/cbb/schools/illinois-state/2019.html
https://www.sports-reference.com/cbb/schools/illinois/2019.html
https://www.sports-reference.com/cbb/schools/incarnate-word/2019.html
https://www.sports-reference.com/cbb/schools/indiana-state/2019.html
https://www.sports-reference.com/cbb/schools/indiana/2019.html
https://www.sports-reference.com/cbb/schools/iona/2019.html
https://www.sports-reference.com/cbb/schools/iowa-state/2019.html
https://www.sports-reference.com/cbb/schools/iowa/2019.html
https://www.sports-reference.com/cbb/schools/ipfw/2019.html
https://www.sports-reference.com/cbb/schools/iupui/2019.html
https://www.sports-reference.com/cbb/schools/jackson-state/2019.html
https://www.sports-reference.com/cbb/schools/jacksonville-state/2019.html
https://www.sports-reference.com/cbb/schools/jacksonville/2019.html
https://www.sports-reference.com/cbb/schools/james-madison/2019.html
https://www.sports-reference.com/cbb/schools/kansas-state/2019.html
https://www.sports-reference.com/cbb/schools/kansas/2019.html
https://www.sports-reference.com/cbb/schools/kennesaw-state/2019.html
https://www.sports-reference.com/cbb/schools/kent-state/2019.html
https://www.sports-reference.com/cbb/schools/kentucky/2019.html
https://www.sports-reference.com/cbb/schools/la-salle/2019.html
https://www.sports-reference.com/cbb/schools/lafayette/2019.html
https://www.sports-reference.com/cbb/schools/lamar/2019.html
https://www.sports-reference.com/cbb/schools/lehigh/2019.html
https://www.sports-reference.com/cbb/schools/liberty/2019.html
https://www.sports-reference.com/cbb/schools/lipscomb/2019.html
https://www.sports-reference.com/cbb/schools/long-beach-state/2019.html
https://www.sports-reference.com/cbb/schools/long-island-university/2019.html
https://www.sports-reference.com/cbb/schools/longwood/2019.html
https://www.sports-reference.com/cbb/schools/louisiana-lafayette/2019.html
https://www.sports-reference.com/cbb/schools/louisiana-monroe/2019.html
https://www.sports-reference.com/cbb/schools/louisiana-state/2019.html
https://www.sports-reference.com/cbb/schools/louisiana-tech/2019.html
https://www.sports-reference.com/cbb/schools/louisville/2019.html
https://www.sports-reference.com/cbb/schools/loyola-il/2019.html
https://www.sports-reference.com/cbb/schools/loyola-marymount/2019.html
https://www.sports-reference.com/cbb/schools/loyola-md/2019.html
https://www.sports-reference.com/cbb/schools/maine/2019.html
https://www.sports-reference.com/cbb/schools/manhattan/2019.html
https://www.sports-reference.com/cbb/schools/marist/2019.html
https://www.sports-reference.com/cbb/schools/marquette/2019.html
https://www.sports-reference.com/cbb/schools/marshall/2019.html
https://www.sports-reference.com/cbb/schools/maryland-baltimore-county/2019.html
https://www.sports-reference.com/cbb/schools/maryland-eastern-shore/2019.html
https://www.sports-reference.com/cbb/schools/maryland/2019.html
https://www.sports-reference.com/cbb/schools/massachusetts-lowell/2019.html
https://www.sports-reference.com/cbb/schools/massachusetts/2019.html
https://www.sports-reference.com/cbb/schools/mcneese-state/2019.html
https://www.sports-reference.com/cbb/schools/memphis/2019.html
https://www.sports-reference.com/cbb/schools/mercer/2019.html
https://www.sports-reference.com/cbb/schools/miami-fl/2019.html
https://www.sports-reference.com/cbb/schools/miami-oh/2019.html
https://www.sports-reference.com/cbb/schools/michigan-state/2019.html
https://www.sports-reference.com/cbb/schools/michigan/2019.html
https://www.sports-reference.com/cbb/schools/middle-tennessee/2019.html
https://www.sports-reference.com/cbb/schools/milwaukee/2019.html
https://www.sports-reference.com/cbb/schools/minnesota/2019.html
https://www.sports-reference.com/cbb/schools/mississippi-state/2019.html
https://www.sports-reference.com/cbb/schools/mississippi-valley-state/2019.html
https://www.sports-reference.com/cbb/schools/mississippi/2019.html
https://www.sports-reference.com/cbb/schools/missouri-kansas-city/2019.html
https://www.sports-reference.com/cbb/schools/missouri-state/2019.html
https://www.sports-reference.com/cbb/schools/missouri/2019.html
https://www.sports-reference.com/cbb/schools/monmouth/2019.html
https://www.sports-reference.com/cbb/schools/montana-state/2019.html
https://www.sports-reference.com/cbb/schools/montana/2019.html
https://www.sports-reference.com/cbb/schools/morehead-state/2019.html
https://www.sports-reference.com/cbb/schools/morgan-state/2019.html
https://www.sports-reference.com/cbb/schools/mount-st-marys/2019.html
https://www.sports-reference.com/cbb/schools/murray-state/2019.html
https://www.sports-reference.com/cbb/schools/navy/2019.html
https://www.sports-reference.com/cbb/schools/nebraska-omaha/2019.html
https://www.sports-reference.com/cbb/schools/nebraska/2019.html
https://www.sports-reference.com/cbb/schools/nevada-las-vegas/2019.html
https://www.sports-reference.com/cbb/schools/nevada/2019.html
https://www.sports-reference.com/cbb/schools/new-hampshire/2019.html
https://www.sports-reference.com/cbb/schools/new-mexico-state/2019.html
https://www.sports-reference.com/cbb/schools/new-mexico/2019.html
https://www.sports-reference.com/cbb/schools/new-orleans/2019.html
https://www.sports-reference.com/cbb/schools/niagara/2019.html
https://www.sports-reference.com/cbb/schools/nicholls-state/2019.html
https://www.sports-reference.com/cbb/schools/njit/2019.html
https://www.sports-reference.com/cbb/schools/norfolk-state/2019.html
https://www.sports-reference.com/cbb/schools/north-alabama/2019.html
https://www.sports-reference.com/cbb/schools/north-carolina-asheville/2019.html
https://www.sports-reference.com/cbb/schools/north-carolina-at/2019.html
https://www.sports-reference.com/cbb/schools/north-carolina-central/2019.html
https://www.sports-reference.com/cbb/schools/north-carolina-greensboro/2019.html
https://www.sports-reference.com/cbb/schools/north-carolina-state/2019.html
https://www.sports-reference.com/cbb/schools/north-carolina-wilmington/2019.html
https://www.sports-reference.com/cbb/schools/north-carolina/2019.html
https://www.sports-reference.com/cbb/schools/north-dakota-state/2019.html
https://www.sports-reference.com/cbb/schools/north-dakota/2019.html
https://www.sports-reference.com/cbb/schools/north-florida/2019.html
https://www.sports-reference.com/cbb/schools/north-texas/2019.html
https://www.sports-reference.com/cbb/schools/northeastern/2019.html
https://www.sports-reference.com/cbb/schools/northern-arizona/2019.html
https://www.sports-reference.com/cbb/schools/northern-colorado/2019.html
https://www.sports-reference.com/cbb/schools/northern-illinois/2019.html
https://www.sports-reference.com/cbb/schools/northern-iowa/2019.html
https://www.sports-reference.com/cbb/schools/northern-kentucky/2019.html
https://www.sports-reference.com/cbb/schools/northwestern-state/2019.html
https://www.sports-reference.com/cbb/schools/northwestern/2019.html
https://www.sports-reference.com/cbb/schools/notre-dame/2019.html
https://www.sports-reference.com/cbb/schools/oakland/2019.html
https://www.sports-reference.com/cbb/schools/ohio-state/2019.html
https://www.sports-reference.com/cbb/schools/ohio/2019.html
https://www.sports-reference.com/cbb/schools/oklahoma-state/2019.html
https://www.sports-reference.com/cbb/schools/oklahoma/2019.html
https://www.sports-reference.com/cbb/schools/old-dominion/2019.html
https://www.sports-reference.com/cbb/schools/oral-roberts/2019.html
https://www.sports-reference.com/cbb/schools/oregon-state/2019.html
https://www.sports-reference.com/cbb/schools/oregon/2019.html
https://www.sports-reference.com/cbb/schools/pacific/2019.html
https://www.sports-reference.com/cbb/schools/penn-state/2019.html
https://www.sports-reference.com/cbb/schools/pennsylvania/2019.html
https://www.sports-reference.com/cbb/schools/pepperdine/2019.html
https://www.sports-reference.com/cbb/schools/pittsburgh/2019.html
https://www.sports-reference.com/cbb/schools/portland-state/2019.html
https://www.sports-reference.com/cbb/schools/portland/2019.html
https://www.sports-reference.com/cbb/schools/prairie-view/2019.html
https://www.sports-reference.com/cbb/schools/presbyterian/2019.html
https://www.sports-reference.com/cbb/schools/princeton/2019.html
https://www.sports-reference.com/cbb/schools/providence/2019.html
https://www.sports-reference.com/cbb/schools/purdue/2019.html
https://www.sports-reference.com/cbb/schools/quinnipiac/2019.html
https://www.sports-reference.com/cbb/schools/radford/2019.html
https://www.sports-reference.com/cbb/schools/rhode-island/2019.html
https://www.sports-reference.com/cbb/schools/rice/2019.html
https://www.sports-reference.com/cbb/schools/richmond/2019.html
https://www.sports-reference.com/cbb/schools/rider/2019.html
https://www.sports-reference.com/cbb/schools/robert-morris/2019.html
https://www.sports-reference.com/cbb/schools/rutgers/2019.html
https://www.sports-reference.com/cbb/schools/sacramento-state/2019.html
https://www.sports-reference.com/cbb/schools/sacred-heart/2019.html
https://www.sports-reference.com/cbb/schools/saint-francis-pa/2019.html
https://www.sports-reference.com/cbb/schools/saint-josephs/2019.html
https://www.sports-reference.com/cbb/schools/saint-louis/2019.html
https://www.sports-reference.com/cbb/schools/saint-marys-ca/2019.html
https://www.sports-reference.com/cbb/schools/saint-peters/2019.html
https://www.sports-reference.com/cbb/schools/sam-houston-state/2019.html
https://www.sports-reference.com/cbb/schools/samford/2019.html
https://www.sports-reference.com/cbb/schools/san-diego-state/2019.html
https://www.sports-reference.com/cbb/schools/san-diego/2019.html
https://www.sports-reference.com/cbb/schools/san-francisco/2019.html
https://www.sports-reference.com/cbb/schools/san-jose-state/2019.html
https://www.sports-reference.com/cbb/schools/santa-clara/2019.html
https://www.sports-reference.com/cbb/schools/savannah-state/2019.html
https://www.sports-reference.com/cbb/schools/seattle/2019.html
https://www.sports-reference.com/cbb/schools/seton-hall/2019.html
https://www.sports-reference.com/cbb/schools/siena/2019.html
https://www.sports-reference.com/cbb/schools/south-alabama/2019.html
https://www.sports-reference.com/cbb/schools/south-carolina-state/2019.html
https://www.sports-reference.com/cbb/schools/south-carolina-upstate/2019.html
https://www.sports-reference.com/cbb/schools/south-carolina/2019.html
https://www.sports-reference.com/cbb/schools/south-dakota-state/2019.html
https://www.sports-reference.com/cbb/schools/south-dakota/2019.html
https://www.sports-reference.com/cbb/schools/south-florida/2019.html
https://www.sports-reference.com/cbb/schools/southeast-missouri-state/2019.html
https://www.sports-reference.com/cbb/schools/southeastern-louisiana/2019.html
https://www.sports-reference.com/cbb/schools/southern-california/2019.html
https://www.sports-reference.com/cbb/schools/southern-illinois-edwardsville/2019.html
https://www.sports-reference.com/cbb/schools/southern-illinois/2019.html
https://www.sports-reference.com/cbb/schools/southern-methodist/2019.html
https://www.sports-reference.com/cbb/schools/southern-mississippi/2019.html
https://www.sports-reference.com/cbb/schools/southern-utah/2019.html
https://www.sports-reference.com/cbb/schools/southern/2019.html
https://www.sports-reference.com/cbb/schools/st-bonaventure/2019.html
https://www.sports-reference.com/cbb/schools/st-francis-ny/2019.html
https://www.sports-reference.com/cbb/schools/st-johns-ny/2019.html
https://www.sports-reference.com/cbb/schools/stanford/2019.html
https://www.sports-reference.com/cbb/schools/stephen-f-austin/2019.html
https://www.sports-reference.com/cbb/schools/stetson/2019.html
https://www.sports-reference.com/cbb/schools/stony-brook/2019.html
https://www.sports-reference.com/cbb/schools/syracuse/2019.html
https://www.sports-reference.com/cbb/schools/temple/2019.html
https://www.sports-reference.com/cbb/schools/tennessee-martin/2019.html
https://www.sports-reference.com/cbb/schools/tennessee-state/2019.html
https://www.sports-reference.com/cbb/schools/tennessee-tech/2019.html
https://www.sports-reference.com/cbb/schools/tennessee/2019.html
https://www.sports-reference.com/cbb/schools/texas-am-corpus-christi/2019.html
https://www.sports-reference.com/cbb/schools/texas-am/2019.html
https://www.sports-reference.com/cbb/schools/texas-arlington/2019.html
https://www.sports-reference.com/cbb/schools/texas-christian/2019.html
https://www.sports-reference.com/cbb/schools/texas-el-paso/2019.html
https://www.sports-reference.com/cbb/schools/texas-pan-american/2019.html
https://www.sports-reference.com/cbb/schools/texas-san-antonio/2019.html
https://www.sports-reference.com/cbb/schools/texas-southern/2019.html
https://www.sports-reference.com/cbb/schools/texas-state/2019.html
https://www.sports-reference.com/cbb/schools/texas-tech/2019.html
https://www.sports-reference.com/cbb/schools/texas/2019.html
https://www.sports-reference.com/cbb/schools/toledo/2019.html
https://www.sports-reference.com/cbb/schools/towson/2019.html
https://www.sports-reference.com/cbb/schools/troy/2019.html
https://www.sports-reference.com/cbb/schools/tulane/2019.html
https://www.sports-reference.com/cbb/schools/tulsa/2019.html
https://www.sports-reference.com/cbb/schools/ucla/2019.html
https://www.sports-reference.com/cbb/schools/utah-state/2019.html
https://www.sports-reference.com/cbb/schools/utah-valley/2019.html
https://www.sports-reference.com/cbb/schools/utah/2019.html
https://www.sports-reference.com/cbb/schools/valparaiso/2019.html
https://www.sports-reference.com/cbb/schools/vanderbilt/2019.html
https://www.sports-reference.com/cbb/schools/vermont/2019.html
https://www.sports-reference.com/cbb/schools/villanova/2019.html
https://www.sports-reference.com/cbb/schools/virginia-commonwealth/2019.html
https://www.sports-reference.com/cbb/schools/virginia-military-institute/2019.html
https://www.sports-reference.com/cbb/schools/virginia-tech/2019.html
https://www.sports-reference.com/cbb/schools/virginia/2019.html
https://www.sports-reference.com/cbb/schools/wagner/2019.html
https://www.sports-reference.com/cbb/schools/wake-forest/2019.html
https://www.sports-reference.com/cbb/schools/washington-state/2019.html
https://www.sports-reference.com/cbb/schools/washington/2019.html
https://www.sports-reference.com/cbb/schools/weber-state/2019.html
https://www.sports-reference.com/cbb/schools/west-virginia/2019.html
https://www.sports-reference.com/cbb/schools/western-carolina/2019.html
https://www.sports-reference.com/cbb/schools/western-illinois/2019.html
https://www.sports-reference.com/cbb/schools/western-kentucky/2019.html
https://www.sports-reference.com/cbb/schools/western-michigan/2019.html
https://www.sports-reference.com/cbb/schools/wichita-state/2019.html
https://www.sports-reference.com/cbb/schools/william-mary/2019.html
https://www.sports-reference.com/cbb/schools/winthrop/2019.html
https://www.sports-reference.com/cbb/schools/wisconsin/2019.html
https://www.sports-reference.com/cbb/schools/wofford/2019.html
https://www.sports-reference.com/cbb/schools/wright-state/2019.html
https://www.sports-reference.com/cbb/schools/wyoming/2019.html
https://www.sports-reference.com/cbb/schools/xavier/2019.html
https://www.sports-reference.com/cbb/schools/yale/2019.html
https://www.sports-reference.com/cbb/schools/youngstown-state/2019.html"""

We need to perform a series of steps to extract the school name element for our tests.

1. Split the blob of text on carriage-return '\n' to get to the individual links
2. Split the link on the '/' character
3. The school name is second element from the end of this list


In [18]:
browser_links.split('\n')[0].split('/')[-2]

'abilene-christian'

Now that we can extract the actual school name segment of the URL we can test our function using the following steps:

1. Create a set of the school names from the text blob
2. Create a set of the function formatted school names in the dataframe
3. Identify any differences


In [19]:
# Create a set of school names from the text blob
url_names = set([s.split('/')[-2] for s in browser_links.split('\n')])

# Create a set of the generated school names in our dataframe
df_names = set(df.school_formatted.unique())

# Drum roll ...
url_names.difference(df_names)

{'arkansas-little-rock',
 'california',
 'california-davis',
 'california-irvine',
 'california-riverside',
 'california-santa-barbara',
 'ipfw',
 'long-beach-state',
 'louisiana-lafayette',
 'nebraska-omaha',
 'southern-illinois-edwardsville',
 'texas-pan-american',
 'virginia-military-institute',
 'william-mary'}

We have 14 schools names which the function did not transform correctly. Let's see what it generated.

In [20]:
df_names.difference(url_names)

{'cal-state-long-beach',
 'little-rock',
 'louisiana',
 'omaha',
 'purdue-fort-wayne',
 'siu-edwardsville',
 'texas-rio-grande-valley',
 'uc-davis',
 'uc-irvine',
 'uc-riverside',
 'uc-santa-barbara',
 'university-of-california',
 'vmi',
 'william--mary'}

The 'william--mary' to 'william-mary' transformation can be corrected in the string operation. The other string transformations can be hard coded.

In [21]:
fixes = { 'cal-state-long-beach': 'long-beach-state',
         'little-rock': 'arkansas-little-rock',
         'louisiana': 'louisiana-lafayette',
         'omaha': 'nebraska-omaha',
         'purdue-fort-wayne': 'ipfw',
         'siu-edwardsville': 'southern-illinois-edwardsville',
         'texas-rio-grande-valley': 'texas-pan-american',
         'uc-davis': 'california-davis',
         'uc-irvine': 'california-irvine',
         'uc-riverside': 'california-riverside',
         'uc-santa-barbara': 'california-santa-barbara',
         'university-of-california': 'california',
         'vmi': 'virginia-military-institute' }


Here's the revised function which should also be tested.

In [8]:
# Revised function
def transform_school_name(s):
    unwanted = "!#$%&'()*+,./:;<=>?@[\]^_`{|}~"
    fixes = { 'cal-state-long-beach': 'long-beach-state',
         'little-rock': 'arkansas-little-rock',
         'louisiana': 'louisiana-lafayette',
         'omaha': 'nebraska-omaha',
         'purdue-fort-wayne': 'ipfw',
         'siu-edwardsville': 'southern-illinois-edwardsville',
         'texas-rio-grande-valley': 'texas-pan-american',
         'uc-davis': 'california-davis',
         'uc-irvine': 'california-irvine',
         'uc-riverside': 'california-riverside',
         'uc-santa-barbara': 'california-santa-barbara',
         'university-of-california': 'california',
         'vmi': 'virginia-military-institute' }
    _s = (s.translate(str.maketrans('', '', unwanted))
          .lower()
          .replace('ncaa', '')
          .replace(' ', '-')
          .replace('--', '-')
          .strip())
    return _s if _s not in fixes.keys() else fixes[_s]


Test the function by applying it to the 'school' column and repeating the steps for the test. This time there should be no differences, i.e. the expected result is an empty set.

In [23]:
# Apply the new function and check the results to be sure
df["school_formatted"] = df.school.apply(transform_school_name)

# Create a new set of the generated school names in our dataframe
df_names = set(df.school_formatted.unique())

# We should have no differences now
url_names.difference(df_names)

set()

## Scrape the 'Basic' and 'Advanced' stats

Now we are ready to scrape the team season stats. The 'basic' and 'advanced' stats pages share the same format so we can use a single function to perform both tasks.

In [ ]:
def create_school_stats_csv_files(seasons, stats="basic"):
    for season in seasons:
        if stats[:3] == "adv":
            url = 'https://www.sports-reference.com/cbb/seasons/{}-advanced-school-stats.html#adv_school_stats::none'.format(season)
        else:
            url = 'https://www.sports-reference.com/cbb/seasons/{}-school-stats.html#basic_school_stats::none'.format(season)
        df = pd.read_html(url)[0]
        # Fix the header
        header = df.columns.tolist()
        cols = [col[1].lower() for col in header]
        df.columns = cols
        # Strip out the dirty rows
        df = df[(df['rk'] != 'Rk') & (df['rk'] == df['rk'])]
        # Add the school_formatted column
        df["school_formatted"] = df["school"]
        # Add season column
        df["season"] = season
        # Apply the school name transformation
        df["school_formatted"] = df.school.apply(transform_school_name)
        # Save to csv file
        df.to_csv("{}_school_stats_{}.csv".format(stats, season), index=False)
        # Give the website a chance to breath
        time.sleep(6)


Create a list of seasons we are interested in the data from and use the function to save the information to csv files.

In [25]:
# Generate the list of seasons we want to scrape
seasons = [2020 - y for y in range(4)]

# Scrape the 'basic' and 'advanced' stats
for stats in ["basic", "adv"]:
    create_school_stats_csv_files(seasons, stats)

We need to be able to merge the contents of these 'Basic' and 'Advanced' stats, so lets create some functions to do so.

In [32]:
def create_merge_id(row):
    # Create a new 'ID' by concatenating the 'school_formatted' and 'season' field contents
    row["ID"] = ",".join([str(row['school_formatted']), str(row['season'])])
    return row

def merge_basic_adv_df(basic_df, adv_df):
    # Remove columns from 'Advanced' that are present in 'Basic'
    adv_df = adv_df.iloc[:, 17:]
    # Create index 'ID' for merge
    basic_df = basic_df.apply(create_merge_id, axis=1)
    adv_df = adv_df.apply(create_merge_id, axis=1)
    # Drop unwanted columns from 'Basic'
    basic_df = basic_df.drop(['rk', 'school', 'school_formatted', 'season'], axis=1)
    # Use the 'ID' column to perform the merge
    lookup_df = basic_df.merge(adv_df, left_on=["ID"], right_on=["ID"], how="left")
    # Remove the 'ID' column as it is no longer needed
    lookup_df = lookup_df.drop(["ID"], axis=1)
    # Rename the 'school_formatted' column to 'team'
    lookup_df.rename(columns={'school_formatted':'team'}, inplace=True)
    # Return the new dataframe
    return lookup_df

Let's test the functions to ensure everything is in order.

In [34]:
BASIC_STATS = os.path.join(os.getcwd(), "basic_school_stats")
ADV_STATS = os.path.join(os.getcwd(), "adv_school_stats")
season = 2019

basic_df = pd.read_csv(os.path.join(BASIC_STATS, "basic_school_stats_{}.csv".format(season)))
adv_df = pd.read_csv(os.path.join(ADV_STATS, "adv_school_stats_{}.csv".format(season)))

merged_df = merge_basic_adv_df(basic_df, adv_df)

merged_df.to_csv("merged_stats_{}.csv".format(season))

merged_df

,g,w,l,w-l%,srs,sos,w.1,l.1,w.2,l.2,...,trb%,ast%,stl%,blk%,efg%,tov%,orb%,ft/fga,team,season
0,34,27,7,0.794,-1.91,-7.34,14,4,13,2,...,50.3,58.5,12.9,8.0,0.535,15.5,28.8,0.239,abilene-christian,2019
1,32,14,18,0.438,-4.28,0.24,8,10,9,6,...,50.1,54.1,7.0,5.8,0.517,17.4,23.7,0.192,air-force,2019
2,33,17,16,0.515,4.86,1.09,8,10,14,3,...,48.2,50.1,8.2,8.9,0.485,15.0,25.3,0.195,akron,2019
3,32,5,27,0.156,-19.23,-8.38,4,14,4,7,...,47.1,52.3,10.7,4.7,0.457,19.4,27.6,0.157,alabama-am,2019
4,35,20,15,0.571,0.36,-1.52,10,8,11,5,...,52.7,44.3,9.3,7.5,0.511,14.8,30.4,0.212,alabama-birmingham,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,35,21,14,0.600,3.29,-0.89,13,5,15,2,...,52.3,54.7,9.0,6.1,0.506,14.6,31.3,0.251,wright-state,2019
349,32,8,24,0.250,-9.75,0.19,4,14,6,10,...,44.7,48.0,7.8,7.9,0.492,18.6,15.6,0.288,wyoming,2019
350,35,19,16,0.543,9.61,8.06,9,9,13,5,...,53.4,56.3,8.1,10.6,0.528,16.5,32.2,0.221,xavier,2019
351,30,22,8,0.733,5.52,-1.24,10,4,11,2,...,52.9,56.3,8.0,11.2,0.556,15.9,25.8,0.227,yale,2019


The output looks good so now we can begin to hoover up the data for the players that make up the teams.

# Team Rosters

The team roster pages provide detailed information which could be useful for developing a model with more fine grained information. The URL for each team roster page can be generated using the 'formatted_school' and the season.

    https://www.sports-reference.com/cbb/schools/{formatted_school}/{season}.html

e.g.

    https://www.sports-reference.com/cbb/schools/abilene-christian/2019.html

## What data do we want?

The roster table can tell us the following:
- Each player's position (Squad Structure)
- Each player's college class (Experience Level)

By knowing the number of players available for each position we can estimate the strength in depth of the team.

The college class of the players can help us determine whether the player is a novice or a veteran without having to do season on season comparisons of the rosters.

## What else can we do with the data?

The player position could be combined with their stats and used to identify the stats that are important for each position. Players who play in the same position would be expected to be clustered together. This process might also reveal different styles of player in each position. This level of information would be beneficial for determining matchups.

## The Process

Let's repeat the process from before.

In [27]:
season = 2019
team = "abilene-christian"
url = "https://www.sports-reference.com/cbb/schools/{}/{}.html#roster::none".format(team, season)

roster_df = pd.read_html(url)[0]
roster_df.head()

,Player,#,Class,Pos,Height,Weight,Hometown,High School,Summary
0,Jaren Lewis,1,SR,F,6-6,215,"Orlando, FL",Lake Highland Preparatory,"13.8 Pts, 6.1 Reb, 2.7 Ast"
1,Payten Ricks,5,JR,G,6-2,190,"Wichita, KS",Maize High School,"11.6 Pts, 2.8 Reb, 1.9 Ast"
2,Jaylen Franklin,0,SR,G,6-2,205,"Jacksonville, AR",Parkview Arts and Science Magnet High School,"10.2 Pts, 3.7 Reb, 4.9 Ast"
3,Jalone Friday,21,JR,C,6-9,235,"Oklahoma City, OK",Oklahoma City Storm Homeschool,"13.1 Pts, 4.7 Reb, 1.0 Ast"
4,Hayden Farquhar,15,SR,F,6-7,215,"Throckmorton, TX",Throckmorton High School,"6.6 Pts, 2.6 Reb, 0.8 Ast"


**Remark:** Height could be of interest so we'll keep it for now.

In [29]:
roster_df = roster_df.iloc[:, 0:5]
roster_df.head()

,Player,#,Class,Pos,Height
0,Jaren Lewis,1,SR,F,6-6
1,Payten Ricks,5,JR,G,6-2
2,Jaylen Franklin,0,SR,G,6-2
3,Jalone Friday,21,JR,C,6-9
4,Hayden Farquhar,15,SR,F,6-7


We certainly don't need the player numer so drop the '#' column.

In [30]:
roster_df = roster_df.drop(['#'], axis=1)
roster_df.head()

,Player,Class,Pos,Height
0,Jaren Lewis,SR,F,6-6
1,Payten Ricks,JR,G,6-2
2,Jaylen Franklin,SR,G,6-2
3,Jalone Friday,JR,C,6-9
4,Hayden Farquhar,SR,F,6-7


Let's take a look at the 'Class' column.

In [31]:
roster_df.Class.unique()

array(['SR', 'JR', 'FR', 'SO'], dtype=object)

A quick Google search tells us the following...

>    For high school and university ("college" in the US), the four years are named:
>    - Freshman for 9th grade or the first year of college,
>    - Sophomore for 10th grade or the second year of college,
>    - Junior for 11th grade or the third year of college,
>    - Senior for 12th grade or the fourth year of college.

**Assumption:** If a player is in their Freshman year (FR) then they have no prior experience, otherwise they are assumed to be experienced players.

In [32]:
# Transform the 'Class' column into an experience indicator 'Exp'
roster_df['Exp'] = roster_df['Class'].map({'FR': 0, 'SO': 1, 'JR': 1, 'SR': 1})

In [40]:
# Add the 'Season' and 'Team' columns to complete the data
roster_df['Season'] = season
roster_df['Team'] = team
roster_df

,Player,Exp,Pos,Height,Season,Team
0,Jaren Lewis,1,F,78,2019,abilene-christian
1,Payten Ricks,1,G,74,2019,abilene-christian
2,Jaylen Franklin,1,G,74,2019,abilene-christian
3,Jalone Friday,1,C,81,2019,abilene-christian
4,Hayden Farquhar,1,F,79,2019,abilene-christian
5,B.J. Maxwell,1,G,76,2019,abilene-christian
6,Joe Pleasant,0,F,80,2019,abilene-christian
7,Damien Daniels,0,G,67,2019,abilene-christian
8,Trey Lenox,1,G,75,2019,abilene-christian
9,Clay Gayman,0,F,78,2019,abilene-christian


Create a function to transform the 'Height' into something we can use. Inches should be fine

In [57]:
def convert_height_to_inches(height):
    if height != height:
        result = -1
    else:
        _h = str(height).split('-')
        if len(_h) == 1:
            result = int(_h[0]) * 12
        else:
            result = (int(_h[0]) * 12 + int(_h[1]))
    return result

In [36]:
roster_df['Height'] = roster_df.Height.apply(convert_height_to_inches)

In [41]:
roster_df

,Player,Exp,Pos,Height,Season,Team
0,Jaren Lewis,1,F,78,2019,abilene-christian
1,Payten Ricks,1,G,74,2019,abilene-christian
2,Jaylen Franklin,1,G,74,2019,abilene-christian
3,Jalone Friday,1,C,81,2019,abilene-christian
4,Hayden Farquhar,1,F,79,2019,abilene-christian
5,B.J. Maxwell,1,G,76,2019,abilene-christian
6,Joe Pleasant,0,F,80,2019,abilene-christian
7,Damien Daniels,0,G,67,2019,abilene-christian
8,Trey Lenox,1,G,75,2019,abilene-christian
9,Clay Gayman,0,F,78,2019,abilene-christian


Now we turn those steps into a function that can be applied across seasons and teams

In [35]:
def create_roster_df(seasons, teams):
    df = pd.DataFrame()
    for season in seasons:
        for team in teams:
            print(team, " : ", datetime.now())
            url = "https://www.sports-reference.com/cbb/schools/{}/{}.html#roster::none".format(team, season)
            roster_df = pd.read_html(url)[0]
            roster_df = roster_df.iloc[:, 0:5]
            roster_df = roster_df.drop(['#'], axis=1)
            # Add Experience as a new column
            roster_df['Exp'] = roster_df['Class'].map({'FR': 0, 'SO': 1, 'JR': 1, 'SR': 1})
            # Add Season
            roster_df['Season'] = season
            # Add Team
            roster_df['Team'] = team
            df = df.append(roster_df, ignore_index=True)
        # Give the server a chance to catch its breath
        time.sleep(5)
    return df


In [62]:
_rosters.to_pickle("./rosters/rosters-2018.pkl")
_rosters.shape

(4741, 7)

In [58]:
_rosters['Height'] = _rosters.Height.apply(convert_height_to_inches)
_rosters.iloc[0][3]

81

Let's take the function for a spin and see where we end up.

In [60]:
_rosters[(_rosters.Height == -1)]

,Player,Class,Pos,Height,Exp,Season,Team
2101,Kiir Deng,JR,F,-1,1,2018,louisiana-monroe
2703,Elijah Cooks,FR,G,-1,0,2018,nevada
2807,Vilontae Eanes,FR,G,-1,0,2018,norfolk-state
2837,Reginald Gardner,FR,G,-1,0,2018,north-carolina-central
4200,Aaron Campbell,SO,F,-1,1,2018,texas-southern


In [66]:
from datetime import datetime

seasons = [2019]
teams = ['south-carolina-state', 'virginia-military-institute', 'kentucky']

rosters_2019 = create_roster_df(seasons, teams)

#rosters_2019.to_pickle("rosters-2019.pkl")

south-carolina-state  :  2020-04-26 05:46:23.122000
virginia-military-institute  :  2020-04-26 05:46:27.375705
kentucky  :  2020-04-26 05:46:30.458734


In [67]:
rosters_2019.shape

(38, 7)

In [ ]:
rosters_2019.columns

## Per 100 Poss

The team roster pages also provide summary performance stats under a number of headings, the one that is of particular interest to us is the 'Per 100 Poss' table. Scraping this data would allow us to perform an Apples to Apples comparison of players. Granted the data will be skewed for players that have very little playing time but we can filter the data to only include players that have played at least 50% of games.

The additional data on the page is hidden inside comments so we need to perform some additional work to scrape it. Time to roll out the Requests and BeautifulSoup libraries.

In [48]:
season = 2019
team = "abilene-christian"
url = 'https://www.sports-reference.com/cbb/schools/{}/{}.html#per_poss_link::none'.format(team, season)

per100_df = pd.DataFrame()

# Extract html from the roster page
req = requests.get(url).text
            
# Create soup object from the html
soup = BeautifulSoup(req, 'html.parser')

0

In [68]:
# Find the <div> tag with the 'id' attribute of 'all_per_poss'
div_all_per_poss = soup.find('div', {'id': 'all_per_poss'})
# Find the enclosed <div> tag with the class of 'placeholder'
placeholder = div_all_per_poss.find('div', {'class': 'placeholder'})
# The next tag is the comment we need to parse
comment = ''.join(placeholder.next_siblings)
# Parse the comment back into soup object
soup_comment = BeautifulSoup(comment, 'html.parser')
# Extract the table from soup object using its 'id' attribute
tables = soup_comment.find_all('table', attrs={'id': 'per_poss'})

In [70]:
len(tables)

1

In [71]:
tables[0].prettify()

'<table class="sortable stats_table" data-cols-to-freeze="2" id="per_poss">\n <caption>\n  Per 100 Poss Table\n </caption>\n <colgroup>\n  <col/>\n  <col/>\n  <col/>\n  <col/>\n  <col/>\n  <col/>\n  <col/>\n  <col/>\n  <col/>\n  <col/>\n  <col/>\n  <col/>\n  <col/>\n  <col/>\n  <col/>\n  <col/>\n  <col/>\n  <col/>\n  <col/>\n  <col/>\n  <col/>\n  <col/>\n  <col/>\n  <col/>\n  <col/>\n  <col/>\n  <col/>\n </colgroup>\n <thead>\n  <tr>\n   <th aria-label="Rank" class="ranker poptip sort_default_asc center" data-stat="ranker" data-tip="Rank" scope="col">\n    Rk\n   </th>\n   <th aria-label="Player" class="poptip sort_default_asc center" data-stat="player" scope="col">\n    Player\n   </th>\n   <th aria-label="Games" class="poptip center" data-stat="g" data-tip="Games" scope="col">\n    G\n   </th>\n   <th aria-label="Games Started" class="poptip center" data-stat="gs" data-tip="Games Started" scope="col">\n    GS\n   </th>\n   <th aria-label="Minutes Played" class="poptip center" data-st

In [73]:
per100_df = pd.read_html(tables[0].prettify())[0]
per100_df.head()

,Rk,Player,G,GS,MP,FG,FGA,FG%,2P,2PA,...,TRB,AST,STL,BLK,TOV,PF,PTS,Unnamed: 24,ORtg,DRtg
0,1,Jalone Friday,23,23,502,13.4,28.3,0.473,10.0,19.2,...,12.7,2.6,1.9,2.6,5.5,8.4,35.7,NaN,103.0,92.2
1,2,Hayden Farquhar,34,6,481,10.4,20.8,0.500,6.2,9.9,...,10.8,3.3,2.7,1.0,3.0,9.0,27.7,NaN,118.8,94.4
2,3,Jaren Lewis,34,34,1056,10.0,21.5,0.466,9.2,19.2,...,11.7,5.2,3.0,1.2,3.4,3.7,26.4,NaN,111.4,93.1
3,4,Kolton Kohl,31,5,142,7.5,16.3,0.462,7.5,16.3,...,21.4,2.1,2.1,5.4,4.6,7.5,24.7,NaN,110.3,87.3
4,5,Clay Gayman,31,2,155,8.4,18.4,0.458,5.4,11.1,...,11.5,3.1,2.7,1.2,6.5,5.0,24.2,NaN,97.7,94.8


Nothing's going to stop us now!

In [74]:
per100_df.columns

Index(['Rk', 'Player', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '2P', '2PA', '2P%',
       '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS', 'Unnamed: 24', 'ORtg', 'DRtg'],
      dtype='object')

In [75]:
# List of columns we want to keep
to_keep = ['Player', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '2P', '2PA', '2P%',
       '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS', 'ORtg', 'DRtg']

# Drop the other columns
for col in per100_df.columns.tolist():
    if col not in to_keep:
        per100_df = per100_df.drop([col], axis=1)

# Add the 'Team' column
per100_df['Team'] = team

per100_df

,Player,G,GS,MP,FG,FGA,FG%,2P,2PA,2P%,...,TRB,AST,STL,BLK,TOV,PF,PTS,ORtg,DRtg,Team
0,Jalone Friday,23,23,502,13.4,28.3,0.473,10.0,19.2,0.519,...,12.7,2.6,1.9,2.6,5.5,8.4,35.7,103.0,92.2,abilene-christian
1,Hayden Farquhar,34,6,481,10.4,20.8,0.500,6.2,9.9,0.625,...,10.8,3.3,2.7,1.0,3.0,9.0,27.7,118.8,94.4,abilene-christian
2,Jaren Lewis,34,34,1056,10.0,21.5,0.466,9.2,19.2,0.482,...,11.7,5.2,3.0,1.2,3.4,3.7,26.4,111.4,93.1,abilene-christian
3,Kolton Kohl,31,5,142,7.5,16.3,0.462,7.5,16.3,0.462,...,21.4,2.1,2.1,5.4,4.6,7.5,24.7,110.3,87.3,abilene-christian
4,Clay Gayman,31,2,155,8.4,18.4,0.458,5.4,11.1,0.483,...,11.5,3.1,2.7,1.2,6.5,5.0,24.2,97.7,94.8,abilene-christian
5,Joe Pleasant,34,2,498,8.1,15.1,0.540,7.6,13.4,0.571,...,10.6,1.2,1.2,0.6,4.4,6.8,22.8,105.5,98.2,abilene-christian
6,Payten Ricks,34,33,1098,6.8,16.1,0.421,2.4,5.0,0.478,...,5.1,3.5,3.1,0.1,2.5,4.1,21.4,114.7,96.1,abilene-christian
7,B.J. Maxwell,24,18,642,7.3,14.9,0.491,4.2,7.1,0.584,...,10.5,3.7,2.4,0.8,1.9,5.9,20.5,128.1,95.0,abilene-christian
8,Chase Cobb,18,0,44,6.8,10.8,0.625,2.7,2.7,1.000,...,5.4,0.0,2.7,0.0,4.1,5.4,20.3,108.3,97.5,abilene-christian
9,Jaylen Franklin,34,33,1092,7.1,13.3,0.535,6.6,12.3,0.542,...,6.9,9.0,3.3,0.4,3.8,4.8,18.9,112.6,94.2,abilene-christian


### ASIDE: Filtering by Minutes Played

From the table we see that the that some players featured in 34 games ('G'), so the maximum minutes played is 1,360 (34 * 40). That being the case only three players played more than half of the possible game minutes.

In [81]:
per100_df[per100_df['MP'] > 680]

,Player,G,GS,MP,FG,FGA,FG%,2P,2PA,2P%,...,TRB,AST,STL,BLK,TOV,PF,PTS,ORtg,DRtg,Team
2,Jaren Lewis,34,34,1056,10.0,21.5,0.466,9.2,19.2,0.482,...,11.7,5.2,3.0,1.2,3.4,3.7,26.4,111.4,93.1,abilene-christian
6,Payten Ricks,34,33,1098,6.8,16.1,0.421,2.4,5.0,0.478,...,5.1,3.5,3.1,0.1,2.5,4.1,21.4,114.7,96.1,abilene-christian
9,Jaylen Franklin,34,33,1092,7.1,13.3,0.535,6.6,12.3,0.542,...,6.9,9.0,3.3,0.4,3.8,4.8,18.9,112.6,94.2,abilene-christian


In [ ]:
A better solution would be to used the top 50% based upon minutes played ('MP').

In [76]:
# For future filtering
per100_df[per100_df['MP'] > np.percentile(per100_df['MP'], 50)]

,Player,G,GS,MP,FG,FGA,FG%,2P,2PA,2P%,...,TRB,AST,STL,BLK,TOV,PF,PTS,ORtg,DRtg,Team
0,Jalone Friday,23,23,502,13.4,28.3,0.473,10.0,19.2,0.519,...,12.7,2.6,1.9,2.6,5.5,8.4,35.7,103.0,92.2,abilene-christian
2,Jaren Lewis,34,34,1056,10.0,21.5,0.466,9.2,19.2,0.482,...,11.7,5.2,3.0,1.2,3.4,3.7,26.4,111.4,93.1,abilene-christian
6,Payten Ricks,34,33,1098,6.8,16.1,0.421,2.4,5.0,0.478,...,5.1,3.5,3.1,0.1,2.5,4.1,21.4,114.7,96.1,abilene-christian
7,B.J. Maxwell,24,18,642,7.3,14.9,0.491,4.2,7.1,0.584,...,10.5,3.7,2.4,0.8,1.9,5.9,20.5,128.1,95.0,abilene-christian
9,Jaylen Franklin,34,33,1092,7.1,13.3,0.535,6.6,12.3,0.542,...,6.9,9.0,3.3,0.4,3.8,4.8,18.9,112.6,94.2,abilene-christian
12,Damien Daniels,34,6,510,4.7,13.2,0.354,2.9,7.0,0.417,...,6.2,7.1,2.3,0.1,3.7,5.5,13.5,97.1,97.6,abilene-christian
13,Trey Lenox,34,8,520,2.9,7.1,0.403,1.0,2.9,0.360,...,4.7,2.5,1.6,0.1,2.2,5.4,8.7,102.1,99.8,abilene-christian


Create a function to perform the 'Per 100 Poss' scrape across seasons and teams.

In [85]:
def per_100_poss_stats_df(seasons, teams):
    df = pd.DataFrame()
    multipleSeasons = False
    if len(seasons) > 1:
        multipleSeasons = True
    for season in seasons:
        for team in teams:
            url = 'https://www.sports-reference.com/cbb/schools/{}/{}.html#per_poss_link::none'.format(team, season)
            # Extract html from the roster page
            req = requests.get(url).text
            # Create soup object from the html
            soup = BeautifulSoup(req, 'html.parser')
            # Find the <div> tag with the 'id' attribute of 'all_per_poss'
            div_all_per_poss = soup.find('div', {'id': 'all_per_poss'})
            # Find the enclosed <div> tag with the class of 'placeholder'
            placeholder = div_all_per_poss.find('div', {'class': 'placeholder'})
            # The next tag is the comment we need to parse
            comment = ''.join(placeholder.next_siblings)
            # Parse the comment back into soup object
            soup_comment = BeautifulSoup(comment, 'html.parser')
            # Extract the table from soup object using its 'id' attribute
            table = soup_comment.find_all('table', attrs={'id': 'per_poss'})
            # Create a dataframe from the table contents
            per100_df = pd.read_html(table[0].prettify())[0]
            # Instead of dropping columns
            per100_df = per100_df[['Player', 'G', 'GS', 'MP', 'FG', 'FGA',
                                   'FG%', '2P', '2PA', '2P%', '3P', '3PA',
                                   '3P%', 'FT', 'FTA', 'FT%', 'TRB', 'AST',
                                   'STL', 'BLK', 'TOV', 'PF', 'PTS', 'ORtg', 'DRtg']]
            # Add the team
            per100_df['Team'] = team
            if multipleSeasons:
                # Distinguish between rosters across seasons
                per100_df['Season'] = season
            df = df.append(per100_df, ignore_index=True)
        # Give the server a chance to catch its breath
        time.sleep(10)
    return df


In [86]:
from datetime import datetime

print("Begin: ", datetime.now())

seasons = [2019]
teams = url_names

player_stats_2019 = per_100_poss_stats_df(seasons, teams)

print("End: ", datetime.now())

Begin:  2020-04-25 17:19:17.971207
End:  2020-04-25 17:25:54.985166


In [88]:
player_stats_2019.shape

(4765, 26)

In [89]:
player_stats_2019.to_pickle("2019stats.pkl")

In [90]:
player_stats_2019.columns

Index(['Player', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '2P', '2PA', '2P%', '3P',
       '3PA', '3P%', 'FT', 'FTA', 'FT%', 'TRB', 'AST', 'STL', 'BLK', 'TOV',
       'PF', 'PTS', 'ORtg', 'DRtg', 'Team'],
      dtype='object')

We will need to merge the 'Per 100 Poss' and 'Team Roster' data in order to cluster the players.

## Merging Team Rosters with Per 100 Poss

In [63]:
# Create a shared key to act as the index for the merge
def create_player_id(row):
    row["ID"] = ",".join([row["Player"], row["Team"], str(row["Season"])])
    return row


def merge_roster_and_stats(roster_df, player_df, season=-1):
    if "Season" not in player_df.columns.tolist():
        player_df["Season"] = season
    # Create the shared ID key
    player_df = player_df.apply(create_player_id, axis=1)
    roster_df = roster_df.apply(create_player_id, axis=1)
    # Drop 'Player', 'Team' and 'Season' keys from the roster dataframe
    roster_df = roster_df.drop(["Player", "Team", "Season"], axis=1)
    # Merge the two dataframes
    df = player_df.merge(roster_df, on="ID", how="left")
    # We don't need 'ID' anymore
    df = df.drop(["ID"], axis=1)
    # Save the merged data
    df.to_pickle("./merged_player_stats/player_stats_{}.pkl".format(str(season)))


In [65]:
_per100_2018 = pd.read_pickle("./per100/stats-2018.pkl")
_rosters_2018 = pd.read_pickle("./rosters/rosters-2018.pkl")

merge_roster_and_stats(_rosters_2018, _per100_2018, 2018)

We now have enough data to begin clustering the players.

## Team Game Logs

The URL for each team game logs page can be generated using the 'formatted_school' and the season.

https://www.sports-reference.com/cbb/schools/{formatted_school}/{season}-gamelogs.html

e.g.

https://www.sports-reference.com/cbb/schools/abilene-christian/2019-gamelogs.html
    

In [2]:
season = 2019
team = "abilene-christian"
url = "https://www.sports-reference.com/cbb/schools/{}/{}-gamelogs.html#sgl-basic::none".format(team, season)

In [6]:
games_df = pd.read_html(url)[0]
games_df.head(25)

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                    G               Date Unnamed: 2_level_1   
0                   1         2018-11-06                NaN   
1                   2         2018-11-09                NaN   
2                   3         2018-11-15                  @   
3                   4         2018-11-22                  N   
4                   5         2018-11-23                  @   
5                   6         2018-11-24                  N   
6                   7         2018-11-27                NaN   
7                   8         2018-12-01                  @   
8                   9         2018-12-04                NaN   
9                  10         2018-12-08                NaN   
10                 11         2018-12-15                  @   
11                 12         2018-12-21                  @   
12                 13         2018-12-29                NaN   
13                 14         2019-01-02                NaN   
14                 15         2019-01-05                  @   
15                 16         2019-01-09                  @   
16                 17         2019-01-12                  @   
17                 18         2019-01-16                NaN   
18                 19         2019-01-19                NaN   
19                 20         2019-01-23                  @   
20                NaN                NaN                NaN   
21                  G               Date                NaN   
22                 21         2019-01-26                NaN   
23                 22         2019-01-30                NaN   
24                 23         2019-02-06                  @   

          Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0  \
                         Opp                W/L                 Tm   
0          Arlington Baptist                  W                107   
1             Arkansas State                  W                 94   
2                     Denver                  W                 67   
3                       Elon                  W                 72   
4                    Pacific                  W                 73   
5               UC-Riverside                  W                 60   
6               Howard Payne                  W                 90   
7                 Pepperdine                  L                 62   
8                   Campbell                  W                 83   
9                  Schreiner                  W                 93   
10                Texas Tech                  L                 48   
11  Southeast Missouri State                  W                 70   
12                   McMurry                  W                 88   
13               New Orleans                  W                 68   
14         Sam Houston State                  L                 68   
15             McNeese State                  W                 73   
16    Southeastern Louisiana                  W                 75   
17           Houston Baptist                  W                 75   
18        Northwestern State                  W                 78   
19         Stephen F. Austin                  L                 60   
20                       NaN                NaN                NaN   
21                       Opp                W/L                 Tm   
22          Central Arkansas                  W                 79   
23  Texas A&M-Corpus Christi                  W                 78   
24                     Lamar                  W                 75   

   Unnamed: 6_level_0  School                  ...  Opponent            \
                  Opp      FG     FGA     FG%  ...        FT       FTA   
0                  54      41      66    .621  ...        13        18   
1                  73      31      55    .564  ...        15        24   
2                  61      22      49    .449  ...         2        11   
3                  56      29      56   

In [9]:
games_df.tail(35)

,G,Date,Unnamed: 2_level_1,Opp,W/L,Tm,Opp,FG,FGA,FG%,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,1,2018-11-06,NaN,Arlington Baptist,W,107,54,41,66,.621,...,13,18,.722,4,17,5,2,0,22,0
1,2,2018-11-09,NaN,Arkansas State,W,94,73,31,55,.564,...,15,24,.625,9,26,13,6,2,16,23
2,3,2018-11-15,@,Denver,W,67,61,22,49,.449,...,2,11,.182,9,31,11,1,4,15,18
3,4,2018-11-22,N,Elon,W,72,56,29,56,.518,...,13,19,.684,5,24,15,3,2,11,15
4,5,2018-11-23,@,Pacific,W,73,71,28,56,.500,...,28,37,.757,8,29,5,6,3,9,19
5,6,2018-11-24,N,UC-Riverside,W,60,48,22,53,.415,...,9,15,.600,9,29,6,2,2,15,21
6,7,2018-11-27,NaN,Howard Payne,W,90,53,37,61,.607,...,9,16,.563,4,20,10,8,1,28,13
7,8,2018-12-01,@,Pepperdine,L,62,77,25,62,.403,...,25,30,.833,5,33,16,6,2,15,16
8,9,2018-12-04,NaN,Campbell,W,83,68,27,48,.563,...,21,28,.750,9,30,6,4,2,22,19
9,10,2018-12-08,NaN,Schreiner,W,93,53,32,55,.582,...,10,14,.714,10,21,5,6,1,20,23


This dataframe has the same issues as we experienced with the roster so we will resolve them in a single step.

In [7]:
# Fix the header
header = games_df.columns.tolist()
cols = [col[1] for col in header]
games_df.columns = cols

# Strip out the dirty rows
games_df = games_df[(games_df['G'] != 'G') & (games_df['G'] == games_df['G'])]

games_df.columns

Index(['G', 'Date', 'Unnamed: 2_level_1', 'Opp', 'W/L', 'Tm', 'Opp', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'Unnamed: 23_level_1', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB', 'AST',
       'STL', 'BLK', 'TOV', 'PF'],
      dtype='object')

In [10]:
# Remove the opponent related columns
games_df = games_df.iloc[:, 0:23]
# Rename the columns
games_df.columns = ['G', 'Date', 'Blank', 'Opp', 'W', 'Score', 'OppScore', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF']
# Transform the 'Opponent'
games_df.Opp = games_df.Opp.apply(transform_school_name)
# Add the 'Team'
games_df["Team"] = team
# Add the 'Season'
games_df["Season"] = season

In [12]:
# Transform the 'W' column to numeric values
games_df["W"] = games_df["W"].astype(str).str[0].map({"W": 1, "L": 0})

In [14]:
# Add 5 game rolling average stats
games_df['Wsum'] = games_df['W'].cumsum(axis=0)
games_df['Wp'] = games_df['Wsum'].astype(int) / games_df['G'].astype(int)
games_df['ppg'] = games_df['Score'].rolling(window=5, center=False).mean()
games_df['pApg'] = games_df['OppScore'].rolling(window=5, center=False).mean()
games_df['FGp'] = games_df['FG%'].rolling(window=5, center=False).mean()
games_df['3Pp'] = games_df['3P%'].rolling(window=5, center=False).mean()
games_df['FTp'] = games_df['FT%'].rolling(window=5, center=False).mean()
games_df['ORBpg'] = games_df['ORB'].rolling(window=5, center=False).mean()
games_df['RBpg'] = games_df['TRB'].rolling(window=5, center=False).mean()
games_df['ASTpg'] = games_df['AST'].rolling(window=5, center=False).mean()
games_df['STLpg'] = games_df['STL'].rolling(window=5, center=False).mean()
games_df['BLKpg'] = games_df['BLK'].rolling(window=5, center=False).mean()
games_df['TOpg'] = games_df['TOV'].rolling(window=5, center=False).mean()
games_df['PFpg'] = games_df['PF'].rolling(window=5, center=False).mean()

In [15]:
games_df.head(20)

,G,Date,Blank,Opp,W,Score,OppScore,FG,FGA,FG%,...,FGp,3Pp,FTp,ORBpg,RBpg,ASTpg,STLpg,BLKpg,TOpg,PFpg
0,1,2018-11-06,NaN,arlington-baptist,1,107,54,41,66,.621,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2018-11-09,NaN,arkansas-state,1,94,73,31,55,.564,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2018-11-15,@,denver,1,67,61,22,49,.449,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2018-11-22,N,elon,1,72,56,29,56,.518,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2018-11-23,@,pacific,1,73,71,28,56,.500,...,0.5304,0.4820,0.6842,8.6,31.2,17.8,6.8,2.8,10.6,22.2
5,6,2018-11-24,N,california-riverside,1,60,48,22,53,.415,...,0.4892,0.4664,0.6772,8.8,30.0,14.8,5.4,1.8,9.6,21.2
6,7,2018-11-27,NaN,howard-payne,1,90,53,37,61,.607,...,0.4978,0.4122,0.6336,8.6,28.0,15.6,7.6,2.4,9.2,18.8
7,8,2018-12-01,@,pepperdine,0,62,77,25,62,.403,...,0.4886,0.3732,0.5940,9.2,28.6,15.0,8.4,1.6,9.6,20.6
8,9,2018-12-04,NaN,campbell,1,83,68,27,48,.563,...,0.4976,0.3510,0.6632,8.2,26.8,13.4,10.4,1.4,10.4,21.0
9,10,2018-12-08,NaN,schreiner,1,93,53,32,55,.582,...,0.5140,0.3892,0.6798,9.0,27.8,15.6,11.6,1.8,12.0,19.4


Unfortunately this data does not distinguish between regular season and tournament games so we will need to figure out a way of doing so. For now we will just hardcode them.

In [40]:
from datetime import date

def add_game_type(row):
    ncaa_dates = {"2013": ((2013,3,19), (2013,4,8)),
                  "2014": ((2014,3,18), (2014,4,7)),
                  "2015": ((2015,3,17), (2015,4,6)),
                  "2016": ((2016,3,15), (2016,4,4)),
                  "2017": ((2017,3,14), (2017,4,3)),
                  "2018": ((2018,3,13), (2018,4,2)),
                  "2019": ((2019,3,19), (2019,4,8))}
    # Safety first, if the season is not in the list don't do anything
    if str(row['Season']) in ncaa_dates.keys():
        # Extract the relevant tournament dates
        ((yb,mb,db), (ye,me,de)) = ncaa_dates[str(row['Season'])]
        # Check if the game takes place within the tournament dates
        if date(yb,mb,db) <= datetime.strptime(row['Date'], '%Y-%m-%d').date() <= date(ye,me,de):
            game_type = "Tour" + str(row['Season'])
        else:
            game_type = "Regular"
        # Set the appropriate value for 'GameType'
        row["GameType"] = game_type
    return row

In [41]:
games_df = games_df.apply(add_game_type, axis=1)
games_df

,G,Date,Blank,Opp,W,Score,OppScore,FG,FGA,FG%,...,3Pp,FTp,ORBpg,RBpg,ASTpg,STLpg,BLKpg,TOpg,PFpg,GameType
0,1,2018-11-06,NaN,arlington-baptist,1,107,54,41,66,.621,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Regular
1,2,2018-11-09,NaN,arkansas-state,1,94,73,31,55,.564,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Regular
2,3,2018-11-15,@,denver,1,67,61,22,49,.449,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Regular
3,4,2018-11-22,N,elon,1,72,56,29,56,.518,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Regular
4,5,2018-11-23,@,pacific,1,73,71,28,56,.500,...,0.4820,0.6842,8.6,31.2,17.8,6.8,2.8,10.6,22.2,Regular
5,6,2018-11-24,N,california-riverside,1,60,48,22,53,.415,...,0.4664,0.6772,8.8,30.0,14.8,5.4,1.8,9.6,21.2,Regular
6,7,2018-11-27,NaN,howard-payne,1,90,53,37,61,.607,...,0.4122,0.6336,8.6,28.0,15.6,7.6,2.4,9.2,18.8,Regular
7,8,2018-12-01,@,pepperdine,0,62,77,25,62,.403,...,0.3732,0.5940,9.2,28.6,15.0,8.4,1.6,9.6,20.6,Regular
8,9,2018-12-04,NaN,campbell,1,83,68,27,48,.563,...,0.3510,0.6632,8.2,26.8,13.4,10.4,1.4,10.4,21.0,Regular
9,10,2018-12-08,NaN,schreiner,1,93,53,32,55,.582,...,0.3892,0.6798,9.0,27.8,15.6,11.6,1.8,12.0,19.4,Regular


This looks good so we'll create a function to perform the 'Game Logs' scrape across seasons and teams.

In [53]:
def get_team_game_logs_df(seasons, teams):
    df = pd.DataFrame()
    for season in seasons:
        for team in teams:
            url = "https://www.sports-reference.com/cbb/schools/{}/{}-gamelogs.html#sgl-basic::none".format(team, season)
            # Get the HTML content
            glogs_df = pd.read_html(url)[0]
            # Fix the header
            header = glogs_df.columns.tolist()
            cols = [col[1] for col in header]
            glogs_df.columns = cols
            # Strip out the dirty rows
            glogs_df = glogs_df[(glogs_df['G'] != 'G') & (glogs_df['G'] == glogs_df['G'])]
            # Remove the opponent related columns
            glogs_df = glogs_df.iloc[:, 0:23]
            # Rename the columns
            glogs_df.columns = ['G', 'Date', 'Blank', 'Opp', 'W', 'Score', 'OppScore', 'FG',
                          'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB',
                          'AST', 'STL', 'BLK', 'TOV', 'PF']
            # Transform the 'Opponent'
            glogs_df.Opp = glogs_df.Opp.apply(transform_school_name)
            # Add the 'Team'
            glogs_df["Team"] = team
            # Add the 'Season'
            glogs_df["Season"] = season
            # Transform the 'W' column to numeric values
            glogs_df["W"] = glogs_df["W"].astype(str).str[0].map({"W": 1, "L": 0})
            # Add 5 game rolling average stats
            glogs_df['Wsum'] = glogs_df['W'].cumsum(axis=0)
            glogs_df['Wp'] = glogs_df['Wsum'].astype(int) / glogs_df['G'].astype(int)
            glogs_df['ppg'] = glogs_df['Score'].rolling(window=5, center=False).mean()
            glogs_df['pApg'] = glogs_df['OppScore'].rolling(window=5, center=False).mean()
            glogs_df['FGp'] = glogs_df['FG%'].rolling(window=5, center=False).mean()
            glogs_df['3Pp'] = glogs_df['3P%'].rolling(window=5, center=False).mean()
            glogs_df['FTp'] = glogs_df['FT%'].rolling(window=5, center=False).mean()
            glogs_df['ORBpg'] = glogs_df['ORB'].rolling(window=5, center=False).mean()
            glogs_df['RBpg'] = glogs_df['TRB'].rolling(window=5, center=False).mean()
            glogs_df['ASTpg'] = glogs_df['AST'].rolling(window=5, center=False).mean()
            glogs_df['STLpg'] = glogs_df['STL'].rolling(window=5, center=False).mean()
            glogs_df['BLKpg'] = glogs_df['BLK'].rolling(window=5, center=False).mean()
            glogs_df['TOpg'] = glogs_df['TOV'].rolling(window=5, center=False).mean()
            glogs_df['PFpg'] = glogs_df['PF'].rolling(window=5, center=False).mean()
            # Add 'GameType' column
            glogs_df = glogs_df.apply(add_game_type, axis=1)
            # Append to the dataframe
            df = df.append(glogs_df, ignore_index=True)
        time.sleep(5)
    return df

In [54]:
seasons = [2019, 2018]
teams = ['south-carolina-state', 'virginia-military-institute', 'kentucky']
test_df = get_team_game_logs_df(seasons, teams)
test_df.shape

(202, 40)

In [60]:
test_df

,G,Date,Blank,Opp,W,Score,OppScore,FG,FGA,FG%,...,3Pp,FTp,ORBpg,RBpg,ASTpg,STLpg,BLKpg,TOpg,PFpg,GameType
0,1,2018-11-06,NaN,brevard,1,99,51,34,57,.596,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Regular
1,2,2018-11-10,@,tulsa,0,52,74,18,58,.310,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Regular
2,3,2018-11-13,@,charleston-southern,0,72,89,29,55,.527,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Regular
3,4,2018-11-16,@,cleveland-state,0,69,84,25,56,.446,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Regular
4,5,2018-11-18,@,ohio-state,0,61,89,23,62,.371,...,0.3102,0.7298,10.2,30.8,9.4,5.8,1.0,13.6,23.2,Regular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,33,2018-03-10,N,alabama,1,86,63,27,42,.643,...,0.4810,0.6596,8.6,32.4,15.2,5.2,3.8,9.8,17.4,Regular
198,34,2018-03-11,N,tennessee,1,77,72,25,50,.500,...,0.4436,0.6762,9.0,31.8,13.6,5.4,3.8,10.0,17.4,Regular
199,35,2018-03-15,N,davidson,1,78,73,26,51,.510,...,0.3548,0.6902,8.0,30.4,12.0,4.8,3.4,9.6,17.8,Tour2018
200,36,2018-03-17,N,buffalo,1,95,75,36,64,.563,...,0.3732,0.7112,7.8,31.8,14.0,5.2,3.8,9.6,18.4,Tour2018


In [56]:
test_df.Team.unique()

array(['south-carolina-state', 'virginia-military-institute', 'kentucky'],
      dtype=object)

In [58]:
test_df.Season.unique()

array([2019, 2018], dtype=int64)

In [59]:
test_df.GameType.unique()

array(['Regular', 'Tour2019', 'Tour2018'], dtype=object)

# Conclusion

Following the recipes in this notebook allows the scraping of data for use in the player classification.